In [4]:
import requests
from packaging.version import Version
from astropy.time import Time
import numpy as np
from pprint import pprint

In [83]:
package = "metomi‑isodatetime"
url = f"https://pypi.org/pypi/{package}/json"
response = requests.get(url)
releases_data = response.json()["releases"]

KeyError: 'releases'

In [28]:
releases: dict[str, dict[str, Time | int]] = {}

for release in releases_data:
    release_data = releases_data[release]

    if not release_data:
        continue

    version = Version(release)
    if version.is_prerelease:
        continue

    # keep yanked releases for now
    # if release_data[0]["yanked"]: continue

    releases[version] = {
        "upload_time": Time(releases_data[release][0]["upload_time"]),
        "major": version.major,
        "minor": version.minor,
        "micro": version.micro,
    }

In [32]:
major_minor_dict: dict[tuple[int, int], set[int]] = {}

for version in releases:
    major = releases[version]["major"]
    minor = releases[version]["minor"]
    micro = releases[version]["micro"]

    if (major, minor) not in major_minor_dict:
        major_minor_dict[(major, minor)] = {micro}
    else:
        major_minor_dict[(major, minor)] |= {micro}

print(major_minor_dict)

{(0, 1): {0, 1}, (0, 2): {0}, (0, 3): {0, 1}, (0, 4): {0}, (0, 5): {0}, (0, 6): {0}, (0, 7): {0}, (0, 8): {1}, (0, 9): {0, 1}, (2023, 1): {0}, (2023, 10): {0}, (2023, 5): {0, 1}, (2024, 10): {0}, (2024, 2): {0}, (2024, 5): {0}, (2024, 7): {0}, (2025, 10): {0}, (2025, 8): {0}}


In [39]:
minor_releases = []

for (major, minor), s in major_minor_dict.items():
    minor_releases.append(f"{major}.{minor}.{min(s)}")
print(minor_releases)

['0.1.0', '0.2.0', '0.3.0', '0.4.0', '0.5.0', '0.6.0', '0.7.0', '0.8.1', '0.9.0', '2023.1.0', '2023.10.0', '2023.5.0', '2024.10.0', '2024.2.0', '2024.5.0', '2024.7.0', '2025.10.0', '2025.8.0']


In [146]:
import functools


class Package:
    def __init__(self, name: str):
        self.name = name
        url = f"https://pypi.org/pypi/{self.name}/json"
        response = requests.get(url)
        self.data = response.json()["releases"]
        self.now = Time.now()

    @functools.cached_property
    def releases(self) -> list[Version]:
        """All releases of the package, excluding prereleases."""
        # epochs are included in the version string, e.g., "1!2025.9.0" to "2!1.0.0".
        # `Version` is able to handle these rare cases, so we do not need to do anything
        # special.  The `cabinet` package is an example on PyPI where epochs are used.

        all_releases: list[str] = sorted(self.data.keys())

        return sorted(
            [
                Version(release)
                for release in all_releases
                if not Version(release).is_prerelease
            ]
        )

    @functools.cached_property
    def release_times(self) -> dict[str, Time]:
        return {
            release: Time(self.data[str(release)][0]["upload_time"])
            for release in self.releases
        }

    @functools.cached_property
    def _epoch_major_minor_dict(self) -> dict[tuple[int, int, int], set[int]]:
        """
        Dictionary where the key is a tuple of the major and minor version numbers,
        and
        """
        epoch_major_minor_dict = {}

        for version in self.releases:
            epoch = version.epoch
            major = version.major
            minor = version.minor
            micro = version.micro

            if (epoch, major, minor) not in major_minor_dict:
                epoch_major_minor_dict[(epoch, major, minor)] = {micro}
            else:
                epoch_major_minor_dict[(epoch, major, minor)] |= {micro}

        return epoch_major_minor_dict

    @functools.cached_property
    def minor_releases(self) -> list[str]:
        """The first release of each major/minor pair."""
        minor_releases = []
        for (epoch, major, minor), micros in self._epoch_major_minor_dict.items():
            minor_releases.append(Version(f"{epoch}!{major}.{minor}.{min(micros)}"))
        return sorted(minor_releases)

    @functools.cached_property
    def months_since_minor_release(self):
        return {
            release: float(
                (self.now - self.release_times[release]).to_value("jd") / 30.25
            )
            for release in self.minor_releases
        }

    def last_supported_release(self, months=24, buffer=3):
        releases = list(self.months_since_minor_release.keys())
        months_since_release = np.array(list(self.months_since_minor_release.values()))

        # get index of the first release that occurred in the last 24 months
        index = next(
            (
                i
                for i, months_ago in enumerate(months_since_release)
                if months_ago < months
            ),
            -1,
        )

        months_ago = months_since_release[index]

        if months_ago < buffer and index > 0:
            index -= 1

        return releases[index]


p = Package("plasmapy")
# print(p.releases)
# print(p._major_minor_dict)
# print(p.releases)
pprint(p.months_since_minor_release)
pprint(p.last_supported_release())

{<Version('0.1.1')>: 89.7315790375838,
 <Version('0.2.0')>: 77.52473942938056,
 <Version('0.3.1')>: 69.39710207094772,
 <Version('0.4.0')>: 63.74121746733586,
 <Version('0.5.0')>: 59.0804782570512,
 <Version('0.6.0')>: 55.94495637152931,
 <Version('0.7.0')>: 47.71938474618495,
 <Version('0.8.1')>: 40.161376634767755,
 <Version('0.9.1')>: 35.7408149573879,
 <Version('2023.1.0')>: 33.795392168896925,
 <Version('2023.5.1')>: 28.988269426319633,
 <Version('2023.10.0')>: 24.529331945450338,
 <Version('2024.2.0')>: 20.95660275965297,
 <Version('2024.5.0')>: 17.88563053743075,
 <Version('2024.7.0')>: 15.470159922187404,
 <Version('2024.10.0')>: 12.130642015851315,
 <Version('2025.8.0')>: 2.8128860470726202,
 <Version('2025.10.0')>: 0.2278768643361647}
<Version('2024.2.0')>


### p.data